In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rand
import math
import threading

from sklearn.preprocessing import MinMaxScaler

In [2]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

np.set_printoptions(precision=6, suppress=True)

In [3]:
real_data = pd.read_excel('nov_nine_var.xlsx', engine='openpyxl')[0:430].to_numpy()
goal_data = pd.read_excel('basic_formula.xlsx', engine='openpyxl').to_numpy()

scaler = MinMaxScaler()
scaler = scaler.fit(real_data[:,1:22])

In [4]:
def argmax(l):
    return max(range(len(l)), key=lambda i: l[i])

def argmin(l):
    return min(range(len(l)), key=lambda i: l[i])

In [5]:
start = tf.constant(scaler.transform(real_data[:,1:22])[-1].reshape(1, 21), dtype=tf.float32)
goal = tf.constant(scaler.transform(goal_data[:,1:22])[argmin(goal_data[:,-1])].reshape(1, 21), dtype=tf.float32)

print(goal[0])
print(start[0])

tf.Tensor(
[0.508362 0.497095 0.492966 0.494638 0.502046 0.500935 0.502494 0.500844
 0.504019 0.502051 0.494209 0.498915 0.504325 0.500418 0.500487 0.49806
 0.496443 0.494332 0.511553 0.498666 0.497106], shape=(21,), dtype=float32)
tf.Tensor(
[0.       0.982134 0.       1.       0.991927 0.996404 0.018404 0.736051
 0.039968 0.378589 0.342857 0.578947 0.25     0.320856 0.223881 0.223404
 1.       0.015124 0.496289 0.564583 1.      ], shape=(21,), dtype=float32)


In [6]:
model_state = tf.constant(scaler.transform(real_data[:,1:22])[-61 : -1].reshape(1, 60, 21), dtype=tf.float32)

In [7]:
# paramater
ACTION_NUM = 5
LENTGTH = 60
EPISODE_DONE = 1000

# learn paramater
TRAIN_FLAG = EPISODE_DONE * 10
LEARN = 1000

MEMORY_SIZE = EPISODE_DONE * LEARN

# dqn paramater
GAMMA = 0.999
EPS_DECAY = 0.0005
BATCH_SIZE = 512

In [8]:
# m = input("one or all?: ") == 'one'
m = False

In [9]:
if m:
    model_list = [
        [
            tf.keras.models.load_model('./model/one_lstm/one_lstm_{0}/{1}_model'.format(j, i)) for i in range(21)
        ]   for j in range(ACTION_NUM)
    ]

else:
    model_list = [
        tf.keras.models.load_model('./model/action_net/action_net{0}'.format(i)) for i in range(5)
    ]

In [10]:
def shift_data(origin, d, one):
    if one:
        shift_d = np.zeros((1, LENTGTH, 21))
        for i in range(21):
            d_s = d[:,i]
            shift_d[0][:,i] = np.concatenate((origin[0][1::][:,i], d_s), axis=0).reshape(1, LENTGTH)
    
    else:
        shift_d = tf.concat((origin[0][1::], d), axis=0).reshape(1, LENTGTH, 21)
    return shift_d

In [11]:
def return_action(idx, s, one):
    if one:
        model_pred = np.zeros((5, 21, 1))
        for i in range(5):
            for j in range(21):
                s_s = s[:,:,j].reshape(1, LENTGTH, 1)
                model_pred[i][j] = model_list[i][j](s_s)[0]
        
        return model_pred[idx].T

    else:
        return model_list[idx](s)

In [12]:
def return_state(s, a):
    ns = a
    return ns

In [13]:
def return_reward(ns, gs):
    ns_s = ns[:,0:21]
    dist = -tf.math.sqrt(tf.math.reduce_sum(tf.math.square(gs - ns_s)))

    end = 100
    equal = tf.math.count_nonzero(ns_s-gs) * -4

    reward = end + equal + dist
    return tf.cast(reward, dtype=tf.float32)

In [14]:
class Memory:  # stored as ( s, a, r, s_ ) in SumTree
    e = 0.01
    a = 0.8
    beta = 0.3
    beta_increment_per_sampling = 0.0005

    def __init__(self, capacity):
        self.tree = SumTree(capacity)
        self.capacity = capacity

    def _get_priority(self, error):
        return (tf.math.abs(error) + self.e) ** self.a

    def add(self, error, sample):
        p = tf.cast(self._get_priority(error), dtype=tf.float32)
        self.tree.add(p, sample)

    def sample(self, n):
        a_is = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        priorities = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)

        state_hist = tf.constant(init_state)
        action_hist = tf.constant([[0]], dtype=tf.int32)
        reward_hist = tf.constant([[0]], dtype=tf.float32)
        next_hist = tf.constant(init_state, dtype=tf.float32)
        done_hist = tf.constant([[0]], dtype=tf.int32)
        
        segment = tf.cast(self.tree.total() / n, tf.float32)

        self.beta = tf.cast(tf.math.minimum(1., self.beta + self.beta_increment_per_sampling), dtype=tf.float32)

        for i in tf.range(n):
            a = tf.cast(segment * i, tf.float32)
            b = tf.cast(segment * (i + 1), tf.float32)

            s = tf.random.uniform(shape=(), minval=a, maxval=b, dtype=tf.float32)
            a_i_p, data = self.tree.get(s)

            priorities = priorities.write(i, a_i_p[1])
            a_is = a_is.write(i, a_i_p[0])

            state_hist = tf.concat((state_hist, data[0]), axis=0)
            action_hist = tf.concat((action_hist, data[1]), axis=0)
            reward_hist = tf.concat((reward_hist, data[2]), axis=0)
            next_hist = tf.concat((next_hist, data[3]), axis=0)
            done_hist = tf.concat((done_hist, data[4]), axis=0)
            
        priorities = priorities.stack()
        a_is = a_is.stack()

        sampling_probabilities = tf.cast(priorities / self.tree.total(), dtype=tf.float32)
        
        is_weight = tf.math.pow(tf.math.multiply(self.tree.n_entries, sampling_probabilities), -self.beta)
        is_weight /= is_weight.max()
        
        batch = (state_hist[1::], action_hist[1::], reward_hist[1::], next_hist[1::], done_hist[1::])
        return batch, a_is, is_weight

    def update(self, a_i, error):
        p = self._get_priority(error)
        self.tree.update(a_i, p)

class SumTree:
    write = 0

    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = tf.Variable(tf.zeros(2 * capacity - 1, dtype=tf.float32))

        self.state_hist = tf.Variable(tf.zeros((capacity, 42), dtype=tf.float32))
        self.action_hist = tf.Variable(tf.zeros((capacity, 1), dtype=tf.int32))
        self.reward_hist = tf.Variable(tf.zeros((capacity, 1), dtype=tf.float32))
        self.next_hist = tf.Variable(tf.zeros((capacity, 42), dtype=tf.float32))
        self.done_hist = tf.Variable(tf.zeros((capacity, 1), dtype=tf.int32))

        self.n_entries = 0

    # update to the root node
    def _propagate(self, a_i, change):
        parent = (a_i - 1) // 2

        self.tree[parent].assign(self.tree[parent]+change)

        if parent != 0:
            self._propagate(parent, change)

    # find sample on leaf node
    def _retrieve(self, a_i, s):
        left = 2 * a_i + 1
        right = left + 1

        if left >= self.tree.shape[0]:
            return a_i

        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s - self.tree[left])

    def total(self):
        return self.tree.shape[0]

    # store priority and sample
    def add(self, p, data):
        a_i = self.write + self.capacity - 1

        self.state_hist[self.write].assign(data[0])
        self.action_hist[self.write].assign(data[1])
        self.reward_hist[self.write].assign(data[2])
        self.next_hist[self.write].assign(data[3])
        self.done_hist[self.write].assign(data[4])
        
        self.update(a_i, p)

        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

        if self.n_entries < self.capacity:
            self.n_entries += 1

    # update priority
    def update(self, a_i, p):
        change = p - self.tree[a_i]

        self.tree[a_i].assign(p)
        self._propagate(a_i, change)

    # get priority and sample
    def get(self, s):
        a_i = self._retrieve(0, s)
        dataa_i = a_i - self.capacity + 1
        
        data = (self.state_hist[dataa_i].reshape(1, 42), self.action_hist[dataa_i].reshape(1, 1), self.reward_hist[dataa_i].reshape(1, 1), self.next_hist[dataa_i].reshape(1, 42), self.done_hist[dataa_i].reshape(1, 1))
        return (a_i, self.tree[a_i]), data

In [15]:
class DQN_Network(tf.keras.models.Model):
    def __init__(self):
        super(DQN_Network, self).__init__()
        self.input_layer = tf.keras.models.Sequential()
        self.input_layer.add(tf.keras.layers.Dense(128, activation='relu'))
        self.input_layer.add(tf.keras.layers.Dense(128, activation='relu'))

        self.q_layer = tf.keras.models.Sequential()
        self.q_layer.add(tf.keras.layers.Dense(128, activation='relu'))
        self.q_layer.add(tf.keras.layers.Dense(128, activation='relu'))
        self.q_layer.add(tf.keras.layers.Dense(1, activation='linear'))

        self.adv_layer = tf.keras.models.Sequential()
        self.adv_layer.add(tf.keras.layers.Dense(128, activation='relu'))
        self.adv_layer.add(tf.keras.layers.Dense(128, activation='relu'))
        self.adv_layer.add(tf.keras.layers.Dense(ACTION_NUM, activation='linear'))
    
    @tf.function
    def call(self, x):
        i = self.input_layer(x)

        q = self.q_layer(i)
        adv = self.adv_layer(i)

        o = q + adv - tf.math.reduce_mean(adv, axis=1, keepdims=True)
        return o

In [41]:
class DQN_Agent:
    def __init__(self):
        self.train_model = self.set_model()
        self.target_model = self.set_model()
        self.target_model.trainable = False

        self.memory = Memory(MEMORY_SIZE)
        self.episode = 1
        self.eps_threshold = 1

        self.optim = tf.keras.optimizers.RMSprop(learning_rate=1e-6)

    def set_model(self):
        net = DQN_Network()
        net.build(input_shape=(1, 42))

        optim = tf.keras.optimizers.RMSprop(learning_rate=1e-6)
        net.compile(optimizer=optim, loss='mse')
        return net

    def update_model(self):
        self.target_model.set_weights(self.train_model.get_weights())

    def soft_update_model(self, tau=0.1):
        train_weight = np.array(self.train_model.get_weights(), dtype=object)
        target_weight = np.array(self.target_model.get_weights(), dtype=object)

        weight = train_weight * tau + target_weight * (1-tau)
        self.target_model.set_weights(weight)

    def memorize(self, cs, a_i, r, ns, d):
        if d and self.memory.tree.n_entries > TRAIN_FLAG:
            self.episode += 1

        td_error = r + GAMMA * tf.argmax(self.target_model(ns)[0]) - tf.argmax(self.train_model(cs)[0])
        data = (cs, a_i, r, ns, d)
        self.memory.add(td_error, data)

    def convert_memory_to_input(self, batch):
        s, a_i, r, ns, d = zip(batch)

        states = tf.convert_to_tensor(s).reshape(BATCH_SIZE, 42)
        action_indexs = tf.convert_to_tensor(a_i).reshape(BATCH_SIZE, 1)
        rewards = tf.convert_to_tensor(r).reshape(BATCH_SIZE, 1)
        next_states = tf.convert_to_tensor(ns).reshape(BATCH_SIZE, 42)
        dones = tf.convert_to_tensor(d).reshape(BATCH_SIZE, 1)

        return states, action_indexs, rewards, next_states, dones

    def act(self, state):
        a_r = self.train_model(state)[0]

        if tf.random.uniform(shape=(), maxval=1, dtype=tf.float32) > self.eps_threshold:
            a_i = tf.argmax(a_r)
            c = 1

        else:
            a_i = int(tf.random.uniform(shape=(), maxval=ACTION_NUM-1))
            c = 0
            
        a_i = tf.cast(a_i, dtype=tf.int32)

        return a_i, c, self.eps_threshold

    def run(self):
        if self.memory.tree.n_entries < TRAIN_FLAG:
            return 1
        
        self.eps_threshold = 0.05 + (1 - 0.05) * math.exp(-1. * self.episode * EPS_DECAY)

        batch, a_is, is_weight = self.memory.sample(BATCH_SIZE)

        states, action_indexs, rewards, next_states, dones = self.convert_memory_to_input(batch)

        is_weight = tf.convert_to_tensor(is_weight)
        loss = self.learn(states, action_indexs, rewards, next_states, dones, is_weight)

        return loss

    @tf.function
    def learn(self, states, action_indexs, rewards, next_states, dones, is_weight):
        with tf.GradientTape() as tape:
            tape.watch(self.train_model.trainable_variables)

            q = self.train_model(states)
            next_q = self.train_model(next_states)
            next_target_q = self.target_model(next_states)

            next_action = tf.argmax(next_q, axis=1).reshape(next_q.shape[0], 1)

            target_val = tf.reduce_sum(tf.one_hot(next_action, ACTION_NUM) * next_target_q, axis=1)
            target_q = rewards + (1 - dones) * GAMMA * target_val

            main_val = tf.reduce_sum(tf.one_hot(action_indexs, ACTION_NUM) * q, axis=1)

            error = tf.square(main_val - target_val) * 0.5
            loss = tf.reduce_mean(error)

        grads = tape.gradient(loss, self.train_model.trainable_weights)
        grads = [(tf.clip_by_value(grad, -1.0, 1.0)) for grad in grads]
        self.optim.apply_gradients(zip(grads, self.train_model.trainable_weights))

        return loss

In [17]:
def env_run(agent, m, max_step, max_episode, init_state, init_model_state):
    global state_hist, action_hist, reward_hist, next_hist, done_hist
    
    for e in range(max_episode):
        state = init_state
        model_state = init_model_state
        steps = 1
        
        while True:
            model_state = shift_data(model_state, state[:,0:21], m)
            a_i, t, eps = agent.act(state)
            action = return_action(a_i, model_state, m)
            
            checker = state[:,0:21] == goal[0]
            if all(checker[0]):
                done = 1
                b = 1
            else:
                done = 0
                b = 0

            next_state = tf.concat((return_state(state, action), goal), axis=1)
            reward = return_reward(next_state, goal)

            if steps == max_step:
                done = 1

            else:
                bbbb = 0

            state = next_state
            steps += 1

            state_hist = tf.concat((state_hist, state), axis=0)
            action_hist = tf.concat((action_hist, tf.constant(a_i).reshape(1, 1)), axis=0)
            reward_hist = tf.concat((reward_hist, tf.constant(reward).reshape(1, 1)), axis=0)
            next_hist = tf.concat((next_hist, next_state), axis=0)
            done_hist = tf.concat((done_hist, tf.constant(done).reshape(1, 1)), axis=0)

#             if done and e % 10 == 0:
            if done:
                print(f'={e}=|{reward}')
                break

In [18]:
def memorize(agent, states, actions, rewards, nexts, dones):
    global save_state, save_reward
    
    save_state = tf.concat((save_state, states), axis=0)
    save_reward = tf.concat((save_reward, rewards), axis=0)

    for i in range(1, len(states)):
        agent.memorize(states[i].reshape(1, 42), actions[i], rewards[i], nexts[i].reshape(1, 42), dones[i])

In [40]:
agent = DQN_Agent()
# agent.train_model.set_weights(tf.keras.models.load_model('./model/dqn_net').get_weights())

init_state = tf.concat((start, goal), axis=1)
init_model_state = model_state

global state_hist, action_hist, reward_hist, next_hist, done_hist

state_hist = tf.constant(tf.zeros((1, 42), dtype=tf.float32))
action_hist = tf.constant(tf.zeros((1, 1), dtype=tf.int32))
reward_hist = tf.constant(tf.zeros((1, 1), dtype=tf.float32))
next_hist = tf.constant(tf.zeros((1, 42), dtype=tf.float32))
done_hist = tf.constant(tf.zeros((1, 1), dtype=tf.int32))

save_state = state_hist
save_reward = reward_hist

max_step = EPISODE_DONE
max_episode = 5
agent.memory.tree.n_entries = TRAIN_FLAG

# env_thread = threading.Thread(target=env_run, args=(agent, m, max_step, max_episode, init_state, init_model_state))
# env_thread.daemon = True

# print("data collection")
# env_thread.start()
# env_thread.join()

env_run(agent, m, max_step, max_episode, init_state, init_model_state)

for ti in range(LEARN):
#     env_thread = threading.Thread(target=env_run, args=(agent, m, max_step, max_episode, init_state, init_model_state))
#     env_thread.daemon = True

    print("data collection")
#     env_thread.start()

    env_run(agent, m, max_step, max_episode, init_state, init_model_state)

#     memorize_thread = threading.Thread(target=memorize, args=(agent, state_hist, action_hist, reward_hist, next_hist, done_hist))
#     memorize_thread.daemon = True
    
    print("memorizing")
    memorize(agent, state_hist, action_hist, reward_hist, next_hist, done_hist)
#     memorize_thread.start()
#     memorize_thread.join()

    state_hist = tf.constant(init_state)
    action_hist = tf.constant(tf.zeros((1, 1), dtype=tf.int32))
    reward_hist = tf.constant(tf.zeros((1, 1), dtype=tf.float32))
    next_hist = tf.constant(init_state)
    done_hist = tf.constant(tf.zeros((1, 1), dtype=tf.int32))
    
    print("run")
    loss = agent.run()
    agent.soft_update_model()

#     env_thread.join()

=0=|14.288034439086914
=1=|14.253286361694336
=2=|14.223055839538574
=3=|14.269937515258789
=4=|14.285579681396484
data collection
=0=|14.285579681396484
=1=|14.223615646362305
=2=|14.223055839538574
=3=|14.223055839538574
=4=|14.285115242004395
memorizing
run
(512, 42) (512, 1) (512, 1) (512, 42) (512, 1)
data collection
=0=|14.282330513000488
=1=|14.252022743225098
=2=|14.285579681396484
=3=|14.253286361694336
=4=|14.223128318786621
memorizing
run
(512, 42) (512, 1) (512, 1) (512, 42) (512, 1)
data collection
=0=|14.269937515258789
=1=|14.285579681396484
=2=|14.223055839538574
=3=|14.223055839538574
=4=|14.288034439086914
memorizing
run
(512, 42) (512, 1) (512, 1) (512, 42) (512, 1)
data collection
=0=|14.222705841064453
=1=|14.253286361694336
=2=|14.288034439086914
=3=|14.285115242004395
=4=|14.222705841064453
memorizing
run
(512, 42) (512, 1) (512, 1) (512, 42) (512, 1)
data collection
=0=|14.222705841064453
=1=|14.268577575683594
=2=|14.252462387084961
=3=|14.268646240234375
=4=|1


KeyboardInterrupt



In [ ]:
===========================================================

In [31]:
agent = DQN_Agent()

In [32]:
agent.train_model(init_state)

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[-0.074353,  0.144177,  0.151431,  0.243113,  0.064228]],
      dtype=float32)>

In [38]:
tf.argmax(agent.train_model(init_state)[0])

TypeError: Got an unexpected keyword argument 'dtype'

In [ ]:
# agent = DQN_Agent()
# # agent.train_model.set_weights(tf.keras.models.load_model('./model/dqn_net').get_weights())

# init_state = tf.concat((start, goal), axis=1)

# state_hist = tf.constant(init_state)
# action_hist = tf.constant(tf.zeros((1, 1), dtype=tf.int32))
# reward_hist = tf.constant(tf.zeros((1, 1), dtype=tf.float32))
# next_hist = tf.constant(init_state)
# done_hist = tf.constant(tf.zeros((1, 1), dtype=tf.int32))

# save_state = state_hist
# save_reward = reward_hist

# for e in range(50000 + TRAIN_FLAG // EPISODE_DONE):
#     state = init_state
#     steps = 1
#     rewards = 0
#     max_reward = -1e+4

#     while True:
# #         state_hist.append(state[0][0:21])
#         model_state = shift_data(model_state, state[:,0:21], m)
#         a_i, t, eps = agent.act(state)
#         action = return_action(a_i, model_state, m)

#         checker = state[:,0:21] == goal[0]
#         if all(checker[0]):
#             done = 1
#         else:
#             done = 0
        
#         next_state = tf.concat((return_state(state, action), goal), axis=1)
#         reward = return_reward(next_state, goal)

#         if abs(reward) > 999 or steps == EPISODE_DONE:
#             bbbb = 1
#             done = 1
#         else:
#             bbbb = 0
#             done = 0

#         state = next_state
#         rewards += reward
#         steps += 1
        
#         state_hist = tf.concat((state_hist, state), axis=0)
#         action_hist = tf.concat((action_hist, tf.constant(a_i).reshape(1, 1)), axis=0)
#         reward_hist = tf.concat((reward_hist, tf.constant(reward).reshape(1, 1)), axis=0)
#         next_hist = tf.concat((next_hist, next_state), axis=0)
#         done_hist = tf.concat((done_hist, tf.constant(done).reshape(1, 1)), axis=0)

#         if max_reward < reward:
#             max_reward = reward

#         if done:
#             rewards = rewards if steps - 1 == EPISODE_DONE else -100
#             print(f'======{e if bbbb else 0} | {reward}, {max_reward}')
#             break
        
#     if e % MEMORIZE_FREQ == 0:
#         save_state = tf.concat((save_state, state_hist), axis=0)
#         save_reward = tf.concat((save_reward, reward_hist), axis=0)
#         print("memorizing")
#         for i in range(1, state_hist.shape[0]):
#             agent.memorize(state_hist[i].reshape(1, 42), action_hist[i], reward_hist[i], next_hist[i].reshape(1, 42), done_hist[i])
#         state_hist = tf.constant(init_state)
#         action_hist = tf.constant(tf.zeros((1, 1), dtype=tf.int32))
#         reward_hist = tf.constant(tf.zeros((1, 1), dtype=tf.float32))
#         next_hist = tf.constant(init_state)
#         done_hist = tf.constant(tf.zeros((1, 1), dtype=tf.int32))

#     if e % LEARN_FREQ == 0:
#         print("learning")
#         loss = agent.run()
#         agent.soft_update_model()

In [ ]:
=============================

In [ ]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [ ]:
state_hist

In [ ]:
plt.plot(dnn_model(state_hist[:,0:21]))
plt.show()